# 4. Searching

## 4.1 Searching Lists

#### Algorithm: Selection
Finding the median of a collection of numbers is selection problem with $k=(n+1)/2$ if $n$ is odd (and, if $n$ is even, the median is the mean of the $k$th and $(k+1)$th smallest items, where $k=n/2$).

#### Initial Insight
Choose a value from $S$, to be used as <b>pivotValue</b>. Then divide the list into two partitions, <b>leftPart</b> (containing the list items that are smaller than <b>pivotValue</b>) and <b>rightPart</b> (containing the list items that are greater than <b>pivotValue</b>).
If the $k$th smallest item has been found, stop. Otherwise, select the partition that must contain the $k$th smallest item, and do the whole thing again with this partition.

#### Specification

<table>
   <tr>
     <th>Name:</th>
     <td><b>Selection</b></td>
   </tr>
   <tr>
     <th>Inputs:</th>
     <td>A sequence of integers $S = \{s_1, s_2, s_3, ..., s_n\}$<br/>An integer $k$</td>
   </tr>
   <tr>
     <th>Outputs:</th>
     <td>An integer $x$</td>
   </tr>
   <tr>
     <th>Preconditions:</th>
     <td>Length of $S>0$ and $k>0$ and $k\le n$</td>
   </tr>
   <tr>
     <th>Postcondition:</th>
     <td>$x$ is the $k$th smallest item in $S$</td>
   </tr>
</table> 

#### Code

In [22]:
def quickSelect(k, aList):

    if len(aList) == 1: 
        return aList[0]     # Base case
    
    pivotValue = aList[0]
    leftPart = []
    rightPart = []
    
    for item in aList[1:]:
        if item < pivotValue: 
            leftPart.append(item)
        else: 
            rightPart.append(item)
    
    if len(leftPart) >= k: 
        return quickSelect(k, leftPart)
    elif len(leftPart) == k - 1: 
        return pivotValue
    else: 
        return quickSelect(k - len(leftPart) -1, rightPart)    



print("Median:", quickSelect(6, [2, 36, 5, 21, 8, 13, 11, 20, 4, 1]))

Median: 11


In [23]:
def quickSelect(k, aList):

    if len(aList) == 1: return aList[0]
    
    pivotValue = aList[0]
    leftPart = [x for x in aList[1:] if x < pivotValue]
    rightPart = [x for x in aList[1:] if not x < pivotValue]

    if len(leftPart) >= k: return quickSelect(k, leftPart)
    elif len(leftPart) == k - 1: return pivotValue
    else: return quickSelect(k - len(leftPart) -1, rightPart)    



print("Median:", quickSelect(6, [2, 36, 5, 21, 8, 13, 11, 20, 4, 1]))

Median: 11


#### Remarks
The crucial step (<i>cf.</i> <b>Quick Sort</b>) that determines whether we have best case or worst case performance is the choice of the pivot – if we are really lucky we will get a value that cuts down the list the algorithm needs to search very substantially at each step.<br/><br/>
The algorithm is divide-and-conquer and each iteration makes the sub-problem substantially smaller. In <b>Quick Sort</b>, both partitions are sorted recursively and provided that the pivot, at each stage, divides the list up into equal parts, we achieve  $O(n $log$ n)$ complexity.<br/><br/>
However, in the <b>Selection</b> algorithm we know which partition to search, so we only deal with one of them on each recursive call and as a result it is even more efficient. Hence, it can be shown that its complexity is $O(n)$.

## 4.2 Searching for patterns

It often happens that we need to search through a string of characters to find an occurrence (if there is one) of a given pattern, e.g. genetics and DNA searches, keyword searches.

### Basic string search

#### Algorithm: StringMatch
We are representing the sequence to be searched simply as a string of characters, referred to as the search string $S$, a shorter sequence is the target string $T$ and we are trying to find where the first occurrence of $T$ is, if it is present in $S$.

#### Initial Insight
Repeatedly shift $T$ one place along $S$ and then compare the characters of $T$ with those of $S$. Do this until a match of $T$ in $S$ is found, or the end of $S$ is reached.

#### Specification

<table>
   <tr>
     <th>Name:</th>
     <td><b>StringMatch</b></td>
   </tr>
   <tr>
     <th>Inputs:</th>
     <td>A search string $S = (s_1, s_2, s_3, ..., s_n)$<br/>A target string $T = (t_1, t_2, t_3, ..., t_m)$</td>
   </tr>
   <tr>
     <th>Outputs:</th>
     <td>An integer $x$</td>
   </tr>
   <tr>
     <th>Preconditions:</th>
     <td>$m\le n$, $m>0$ and $n>0$</td>
   </tr>
   <tr>
     <th>Postcondition:</th>
     <td>If there is an occurrence of $T$ in $S$, $x$ is the start position of the first occurrence of $T$ in $S$; otherwise $x = -1$</td>
   </tr>
</table> 

#### Code

In [85]:
def basicStringSearch(searchString, target):

    searchIndex = 0
    
    lenT = len(target)    
    lenS = len(searchString)    
    
    while searchIndex + lenT <= lenS:

        targetIndex = 0

        while targetIndex < lenT and target[targetIndex] == searchString[ targetIndex + searchIndex]:
            targetIndex += 1

        if targetIndex == lenT:
            return searchIndex

        searchIndex += 1

    return -1

In [86]:
# Test Code
for target, index in [('per', 0), ('lta', 14), ('ad', 10), ('astra', -1)]:
    print(basicStringSearch('per ardua ad alta', target)==index)

True
True
True
True


#### Remarks
It becomes immediately apparent when implement that this algorithm would consist of two nested loops leading to complexity $O(mn) > O(m^2)$.<br/><br/>
We know that if the character in $S$ following the failed comparison with $T$ is not in $T$ then there is no need to slide along one place to do another comparison. We should slide to the next point beyond it. This gives us the basis for an improved algorithm.

### Quick search

#### Initial Insight
For each character in $T$ calculate the number of positions to shift $T$ if a comparison fails, according to where (if at all) that character appears in $T$.<br/><br/>
Repeatedly compare the characters of $T$ with those of $S$. If a comparison fails, examine the next character along in $S$ and shift $T$ by the calculated shift distance for that character.<br/><br/>
Do this until an occurrence of $T$ in $S$ is found, or the end of $S$ is reached.

#### Remarks
An important point to note first of all is that the part of the algorithm calculating the shifts depends entirely on an analysis of the target string $T$ – there is no need to examine the search string $S$ at all because for any character in $S$ that is not in $T$, the shift is a fixed distance.<br/><br/>
The database is called a <b>shift table</b> and it stores a <b>shift distance</b> for each character in the domain of $S$ – e.g. for each character of the alphabet, or say, all upper and lower case plus punctuation.<br/><br/>
The <b>shift distance</b> is calculated according to the following rules:
<ol>
  <li>If the character does not appear in T, the shift distance is one more than the length of T.</li>
  <li>If the character does appear in T, the shift distance is the first position at which it appears, counting from right to left and starting at 1. (Hence when a character appears more than once in $T$ keeps the lowest position.)</li>
</ol>

Suppose $S = $'GGGGGAGGCGGCGGT'. Then for target string $T = $'TCCACC', we have:
<table>
   <tr>
     <th>G</th>
     <th>A</th>
     <th>C</th>
     <th>T</th>
   </tr>
   <tr>
     <td>7</td>
     <td>3</td>
     <td>1</td>
     <td>6</td>
   </tr>
</table>
and if $T = $'TGGCG', we have:
<table>
   <tr>
     <th>G</th>
     <th>A</th>
     <th>C</th>
     <th>T</th>
   </tr>
   <tr>
     <td>1</td>
     <td>6</td>
     <td>2</td>
     <td>5</td>
   </tr>
</table>

<br/>
Once the shift table has been computed, the search part of the quick search algorithm is similar to the basic string search algorithm, except that at the end of each failed attempt we look at the next character along in $S$ that is beyond $T$ and use this to look up in the shift table how many steps to slide $T$.<br/>


We implement the <b>shift table</b> as a dictionary in Python:

#### Code

In [87]:
def buildShiftTable(target, alphabet):

    shiftTable = {}

    for character in alphabet:
        shiftTable[character] = len(target) + 1

    for i in range(len(target)):
        char = target[i]
        shift = len(target) - i
        shiftTable[char] = shift

    return shiftTable

In [88]:
def quickSearch (searchString, target, alphabet):

    shiftTable = buildShiftTable(target, alphabet)
    searchIndex = 0

    while searchIndex + len(target) <= len(searchString):
 
        targetIndex = 0

        # Compares the strings    
        while targetIndex < len(target) and target[targetIndex] == searchString[searchIndex + targetIndex]:
            targetIndex = targetIndex + 1

        # Return index if target found
        if targetIndex == len(target): return searchIndex

        # Continue search with new shivt value or exit
        if searchIndex + len(target) < len(searchString):
            next = searchString[searchIndex + len(target)]
            shift = shiftTable[next]
            searchIndex = searchIndex + shift
        else:
            return -1

    return -1

#### Tests

In [89]:
theAlphabet = {'G', 'A', 'C', 'T'}
stringToSearch = 'ATGAATACCCACCTTACAGAAACCTGGGAAAAGGCAATAAATATTATAAAAGGTGAACTTACAGAAGTAA'

for thetarget in ['ACAG', 'AAGTAA', 'CCCC']:
    print(quickSearch(stringToSearch, thetarget, theAlphabet))

15
64
-1


#### Remarks
The basic brute-force algorithm we wrote first will work fine with relatively short search strings but, as with all algorithms, inputs of huge size may overwhelm it. For example, DNA strings can be billions of bases long, so algorithmic efficiency can be vital. We noted already that the complexity of the basic string search can be as bad as O(nm) in the worst case.<br/><br/>

As for the quick search algorithm, research has shown that its average-case performance is good but, unfortunately, its worst case behaviour is still O(mn).<br/><br/>

### Knuth–Morris–Pratt (KMP)

Better algorithms have been developed. One of the best-known efficient search algorithms is the <b>Knuth–Morris–Pratt (KMP)</b> algorithm. A full description of the precise details of the KMP algorithm is beyond the scope of this text.

#### Algorithm: Knuth–Morris–Pratt (KMP)
The <b>KMP</b> algorithm is in two parts:

<ol>
  <li>Build a table of the lengths of prefix matches up to every character in the target string, $T$.</li>
  <li>Move along the search string, $S$, using the information in the table to do the shifting and compare.</li>
</ol>

Once the prefix table has been built, the actual search in the second step proceeds like the other string-searching algorithms above, but when a mismatch is detected the algorithm uses the prefix table to decide how to shift $T$. The problem is to know if these prefix matches exist and – if they do – how long the matching substrings are.</br>

The prefix will then be aligned as shown in Figure 4.17 and comparison can continue at the next character in S.

If you want to take the trouble, you can verify that the final table will be:

In [100]:
prefixTable = [0, 1, 0, 0, 0, 1, 2, 3, 4, 0, 0, 0, 1, 2]

#### Code

In [92]:
# Helper function for kmpSearch()

def buildPrefixTable(target):    

    #The first line of code just builds a list that has len(target)
    #items all of which are given the default value 0

    prefixTable = [0] * len(target)
    q = 0

    for p in range(1, len(target)):

        while q > 0 and target[q] != target[p]:
            q = prefixTable[q - 1]

        if target[q] == target[p]:
            q = q + 1
        
        prefixTable[p] = q

    return prefixTable

In [93]:
def kmpSearch(searchString, target):

    n = len(searchString)
    m = len(target)
    prefixTable = buildPrefixTable(target)
    q = 0

    for i in range(n):

        while q > 0 and target[q] != searchString[i]:
            q = prefixTable[q - 1]

        if target[q] == searchString[i]:
            q = q + 1
    
        if q == m:
            return i - m + 1

    return -1

#### Tests

In [94]:
stringToSearch = 'ATGAATACCCACCTTACAGAAACCTGGGAAAAGGCAATAAATATTATAAAAGGTGAACTTACAGAAGTAA'

for thetarget in ['ACAG', 'AAGTAA', 'CCCC']:
    print(kmpSearch(stringToSearch, thetarget))

15
64
-1


#### Remarks
What about the complexity of the KMP algorithm? Computing the prefix table takes significant effort but in fact there is an efficient algorithm for doing it. Overall, the KMP algorithm has complexity $O(m + n)$. Since $n$ is usually enormously larger than $m$ (think of searching a DNA string of billions of bases), $m$ is usually dominated by $n$, so this means that KMP has effective complexity $O(n)$.

### Other Algorithms

String search is an immensely important application in modern computing, and at least 30 efficient algorithms have been developed for the task. Many of these depend on the principle embodied in the quick search and KMP algorithms – shifting the target string an appropriate distance along the search string at each step, based on information in a table. The <b>Boyer–Moore</b> algorithm, for example, combines elements of both these two algorithms. This algorithm is widely used in practical applications.

There are also string-search algorithms that work in entirely different ways from the examples we have looked at. Generally, these are beyond the scope of this text, but some are based on hashing functions, which we now move on to discuss next.

## 4.3 Hashing and Hash Tables